In [1]:
import torch, torchvision
import torch.nn as nn

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

import detectron2
from detectron2.modeling import BACKBONE_REGISTRY, Backbone, ShapeSpec
import detectron2.data.transforms as T
from detectron2.data.transforms import Transform
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper, build_detection_train_loader, build_detection_test_loader
from detectron2.modeling import build_model,build_resnet_backbone,build_backbone
from detectron2.structures import ImageList, Instances
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
from detectron2.evaluation import COCOEvaluator
from detectron2.data.datasets import register_coco_instances
from segmentation_models_pytorch.encoders import get_encoder

from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

## Blur Filter: Try the Gaussian blur filters to smooth the image and reduce the noise

In [2]:
from transformations import GaussianBlur

In [3]:
def build_sem_seg_train_aug(cfg):
    """
    Define a list of augmentations to apply to the images.
    """
    
    augs= [GaussianBlur()]
    
    return augs

In [4]:
register_coco_instances("SegPC_train", {}, "../TCIA_SegPC_dataset/coco/COCO.json", "../TCIA_SegPC_dataset/coco/x/")
register_coco_instances("SegPC_val", {}, "../TCIA_SegPC_dataset/coco_val/COCO.json", "../TCIA_SegPC_dataset/coco_val/x/")

train_meta = MetadataCatalog.get('SegPC_train')
val_meta = MetadataCatalog.get('SegPC_val')

train_dicts = DatasetCatalog.get("SegPC_train")
val_dicts = DatasetCatalog.get("SegPC_val")

[04/28 21:49:41 d2.data.datasets.coco]: Loaded 298 images in COCO format from ../TCIA_SegPC_dataset/coco/COCO.json
[04/28 21:49:41 d2.data.datasets.coco]: Loaded 200 images in COCO format from ../TCIA_SegPC_dataset/coco_val/COCO.json


In [5]:
@BACKBONE_REGISTRY.register()
class Effb5(Backbone):
    def __init__(self, cfg, input_shape):
        super().__init__()
        
        # Set up the illumination layer
#         self.illumination = nn.Conv2d(in_channels=3, out_channels=9, kernel_size=1, stride=1, padding=0, bias=False)
#         torch.nn.init.normal_(self.illumination.weight, mean=0.0, std=0.05)
#         in_channels = 1
        
        in_channels = 3
        encoder_name = 'timm-efficientnet-b5'
        encoder_depth = 5
        encoder_weights = 'noisy-student'
        self.encoder = get_encoder(encoder_name,
                in_channels=in_channels,
                depth=encoder_depth,
                weights=encoder_weights)
        self.channels = self.encoder.out_channels
        self.conv = nn.ModuleList(
            [nn.Conv2d(self.channels[i],256,3,stride = 2, padding = 1) for i in range(len(self.channels))]
        )

        self.names = ["p"+str(i+1) for i in range(6)]
        
    def forward(self, image):

#         illuminated_image = torch.sum(self.illumination(image), dim=1, keepdim=True)
#         features = self.encoder(illuminated_image)
        features = self.encoder(image)
        out = {self.names[i]: self.conv[i](features[i]) for i in range(1, len(features))}

        return out
    def output_shape(self):
        out_shape = {self.names[i]: ShapeSpec(channels =256, stride = 2**(i+1)) for i in range(1, len(self.names))}
        return out_shape

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
cfg.DATASETS.TRAIN = ("SegPC_train",)
cfg.DATASETS.TEST = ("SegPC_val",)

cfg.DATALOADER.NUM_WORKERS = 5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02/8
cfg.SOLVER.LR_SCHEDULER_NAME = 'WarmupCosineLR'

cfg.SOLVER.WARMUP_ITERS = 100
cfg.SOLVER.MAX_ITER = 3725
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.CHECKPOINT_PERIOD = 500

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.TEST.EVAL_PERIOD = 250
cfg.MODEL.BACKBONE.NAME = "Effb5"

cfg.CUDNN_BENCHMARK = True
cfg.OUTPUT_DIR = "./output/"

In [7]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        
        mapper = DatasetMapper(cfg, is_train=True, augmentations=build_sem_seg_train_aug(cfg))
        
        return build_detection_train_loader(cfg, mapper=mapper)
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        
        mapper = DatasetMapper(cfg, is_train=False, augmentations=build_sem_seg_train_aug(cfg))
        
        return build_detection_test_loader(cfg, dataset_name, mapper=mapper)
    

In [9]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[04/28 21:49:48 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): Effb5(
    (encoder): EfficientNetEncoder(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): Swish()
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNormAct2d(
              48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Swish()
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): Swish()
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
       

Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.2.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) 

[04/28 21:49:49 d2.engine.train_loop]: Starting training from iteration 0


/hpc/home/ak704/.local/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/28 21:50:01 d2.utils.events]:  eta: 0:35:51  iter: 19  total_loss: 4.093  loss_cls_stage0: 0.5787  loss_box_reg_stage0: 0.7036  loss_cls_stage1: 0.5336  loss_box_reg_stage1: 0.4116  loss_cls_stage2: 0.5618  loss_box_reg_stage2: 0.181  loss_mask: 0.6898  loss_rpn_cls: 0.207  loss_rpn_loc: 0.1381  time: 0.5812  data_time: 0.0371  lr: 0.00047618  max_mem: 26034M
[04/28 21:50:13 d2.utils.events]:  eta: 0:35:39  iter: 39  total_loss: 3.446  loss_cls_stage0: 0.4811  loss_box_reg_stage0: 0.9509  loss_cls_stage1: 0.3266  loss_box_reg_stage1: 0.5341  loss_cls_stage2: 0.1846  loss_box_reg_stage2: 0.1922  loss_mask: 0.6089  loss_rpn_cls: 0.06208  loss_rpn_loc: 0.05189  time: 0.5811  data_time: 0.0135  lr: 0.00097479  max_mem: 26034M
[04/28 21:50:25 d2.utils.events]:  eta: 0:35:33  iter: 59  total_loss: 3.392  loss_cls_stage0: 0.475  loss_box_reg_stage0: 0.9721  loss_cls_stage1: 0.3381  loss_box_reg_stage1: 0.6  loss_cls_stage2: 0.1918  loss_box_reg_stage2: 0.2815  loss_mask: 0.4406  loss_rpn_

In [ ]:
# from detectron2.utils.visualizer import Visualizer, ColorMode

# for d in random.sample(train_dicts, 3):
#     print(d["file_name"])
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=train_meta, instance_mode=ColorMode.IMAGE_BW, scale=1)
#     out = visualizer.draw_dataset_dict(d)
#     plt.imshow(out.get_image()[:, :, ::-1])